# Evaluation

Steps:
1. Harmonize the predictions to have the same format
2. Extract the frequencies using a sliding window approach
3. Evaluate the performance of the models
4. Visualize the results

## Step 1: Harmonize the predictions

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
raft_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
raft_predictions = pd.read_csv(raft_file)
raft_predictions['signal'] = raft_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
raft_predictions = raft_predictions[raft_predictions['roi'] == 'chest']

# Only keep the columns that are needed
raft_predictions = raft_predictions[['subject', 'setting', 'model', 'signal']]

raft_predictions.head()

In [ ]:
flownet_file = utils.join_paths(signals_dir, 'flownet_predictions.csv')
flownet_predictions = pd.read_csv(flownet_file)
flownet_predictions['signal'] = flownet_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
flownet_predictions = flownet_predictions[flownet_predictions['roi'] == 'chest']

# Only keep the columns that are needed
flownet_predictions = flownet_predictions[['subject', 'setting', 'model', 'signal']]

flownet_predictions.head()

In [ ]:
pretrained_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
pretrained_predictions = pd.read_csv(pretrained_file)
pretrained_predictions['signal'] = pretrained_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
pretrained_predictions = pretrained_predictions[['subject', 'setting', 'model', 'signal']]

pretrained_predictions.head()

In [ ]:
lucas_kanade_file = utils.join_paths(signals_dir, 'lucas_kanade.csv')
lucas_kanade = pd.read_csv(lucas_kanade_file)
lucas_kanade['signal'] = lucas_kanade['signal'].apply(eval).apply(np.array)

# Rename column method to model
lucas_kanade.rename(columns={'method': 'model'}, inplace=True)

# Remove all the rows that have a signal with a length of 0
lucas_kanade = lucas_kanade[lucas_kanade['grey'] == False]

# Only keep the columns that are needed
lucas_kanade = lucas_kanade[['subject', 'setting', 'model', 'signal']]

lucas_kanade.head()

In [ ]:
pixel_intensity_file = utils.join_paths(signals_dir, 'pixel_intensity.csv')
pixel_intensity = pd.read_csv(pixel_intensity_file)
pixel_intensity['signal'] = pixel_intensity['signal'].apply(eval).apply(np.array)

# Rename column method to model
pixel_intensity.rename(columns={'method': 'model'}, inplace=True)

# Only keep the columns that are needed
pixel_intensity = pixel_intensity[['subject', 'setting', 'model', 'signal']]

pixel_intensity.head()

In [ ]:
r_ppg_path = utils.join_paths(signals_dir, 'r_ppg_predictions.csv')

r_ppg_prediction = pd.read_csv(r_ppg_path)
r_ppg_prediction['signal'] = r_ppg_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
r_ppg_prediction = r_ppg_prediction[['subject', 'setting', 'model', 'signal']]
r_ppg_prediction.head()

In [ ]:
transformer_path = utils.join_paths(signals_dir, 'transformer_predictions.csv')

transformer_prediction = pd.read_csv(transformer_path)
transformer_prediction['signal'] = transformer_prediction['signal'].apply(eval).apply(np.array)

# Add a tf_ prefix to the model names
transformer_prediction['model'] = 'tf_' + transformer_prediction['model']

# Only keep the columns that are needed
transformer_prediction = transformer_prediction[['subject', 'setting', 'model', 'signal']]
transformer_prediction.head()

In [ ]:
#
# The random signal is used as a baseline to see how well the models perform against a random predictions
#
random_path = utils.join_paths(signals_dir, 'random_predictions.csv')

random_prediction = pd.read_csv(random_path)
random_prediction['signal'] = random_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
random_prediction = random_prediction[['subject', 'setting', 'model', 'signal']]
random_prediction.head()

In [ ]:
rhythm_former_path = utils.join_paths(signals_dir, 'rhythm_former.csv')

rhythm_former = pd.read_csv(rhythm_former_path)
rhythm_former['signal'] = rhythm_former['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
rhythm_former = rhythm_former[['subject', 'setting', 'model', 'signal']]
rhythm_former.head()

In [ ]:
predictions = pd.concat([
    raft_predictions,
    flownet_predictions,
    pretrained_predictions,
    lucas_kanade,
    pixel_intensity,
    r_ppg_prediction,
    transformer_prediction,
    random_prediction,
    rhythm_former,
])
len(predictions)

In [ ]:
# Show all models
predictions['model'].unique()

## Step 2: Extract the frequencies using a sliding window approach

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
subject = 'Proband22'
setting = '101_natural_lighting'

In [ ]:
sampling_rate = 30
lowpass = 0.1
highpass = 0.5

In [ ]:
from respiration.analysis import (
    butterworth_filter,
    normalize_signal,
    detrend_tarvainen,
)

from scipy.signal import detrend

models = [
    # 'lucas_kanade',
    'RF_20240802_155121',
    # 'RF_20240726_104536',
    # 'raft_small',
    # 'pixel_intensity_grey',
    # 'tf_20240729_195756',
    # 'MMPD_intra_RhythmFormer',
    # 'mtts_can',
    # 'big_small',
]

signals = []

for model in models:
    prediction_x = predictions[
        (predictions['subject'] == subject) &
        (predictions['setting'] == setting) &
        (predictions['model'] == model)].iloc[0]['signal']

    print(f'{model}: {prediction_x.shape}')

    # Normalize the signals
    prediction = normalize_signal(prediction_x)

    # Filter the signals
    prediction = butterworth_filter(prediction, sampling_rate, lowpass, highpass)

    # Add the signals to the list
    signals.append({
        'label': model,
        'signal': prediction,
        'signal_x': prediction_x,
    })

In [ ]:
# Get the ground truth signal
gt_signal_xxx = dataset.get_breathing_signal(subject, setting)
gt_signal = dataset.get_breathing_signal(subject, setting)
gt_signal = normalize_signal(gt_signal)
gt_signal = butterworth_filter(gt_signal, sampling_rate, lowpass, highpass)

In [ ]:
# Plot the gt_signal_xxx signal
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 5))
plt.plot(gt_signal_xxx)
plt.title('Ground truth signal')
plt.show()

In [ ]:
from respiration.analysis import frequency_from_psd

gt_freq = frequency_from_psd(gt_signal, sampling_rate)
print(f'Ground truth signal {gt_freq:.2f} ({gt_freq * 60:.2f} bpm)')

In [ ]:
# Plot the ground truth signal spectrogram
from scipy.signal import spectrogram
import matplotlib.pyplot as plt

f, t, Sxx = spectrogram(
    gt_signal,
    fs=sampling_rate,
    nperseg=200,
    # window=('tukey', 5.0),
    # mode="magnitude",
)
print(f'f.shape: {f.shape}')
print(f't.shape: {t.shape}')
print(f'Sxx.shape: {Sxx.shape}')

plt.figure(figsize=(20, 5))
# plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
plt.pcolormesh(t, f, Sxx)
# plt.pcolormesh(t, f, Sxx, cmap='viridis')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

# Show the range 0 - 1 Hz
plt.ylim(0, 0.6)

# Add a grid
# plt.grid()

plt.title('Spectrogram of the ground truth signal')
plt.show()

In [ ]:
# Plot the max Sxx value
max_Sxx = np.max(Sxx, axis=0)

# Replace the max values with the frequency
values = np.array([f[np.argmax(Sxx[:, i])] for i in range(len(max_Sxx))])

print(f'Mean: {np.mean(values)}')

plt.figure(figsize=(20, 5))
plt.plot(t, values)
plt.title('Max Sxx value')
plt.ylim(0, 0.6)
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Generate a sample signal (e.g., a sine wave)
gt_signal_torch = torch.Tensor(gt_signal.copy())

# Step 2: Define parameters for the STFT
n_fft = 256  # Number of FFT points
hop_length = 30  # Number of samples between successive frames (overlap)
win_length = n_fft  # Window length
window = torch.hann_window(win_length)  # Hanning window

stft = torch.stft(
    gt_signal_torch,
    n_fft=n_fft,
    hop_length=hop_length,
    win_length=win_length,
    window=window,
    return_complex=True,
)
print('stft.shape', stft.shape)

frequencies = torch.fft.fftfreq(n_fft, 1 / sampling_rate)[:n_fft // 2 + 1]  # Only keep non-negative frequencies
print('frequencies.shape', frequencies.shape)

times = torch.arange(stft.size(1)) * hop_length / sampling_rate
print('times.shape', times.shape)

spectrogram = torch.abs(stft)
print('spectrogram.shape', spectrogram.shape)

# Convert to dB scale (optional)
spectrogram_db = 10 * torch.log10(spectrogram + 1e-10)  # Add a small value to avoid log(0)

# Step 5: Plot the spectrogram
plt.figure(figsize=(20, 5))
# plt.pcolormesh(spectrogram.numpy(), shading='gouraud')
plt.pcolormesh(times.numpy(), frequencies.numpy(), spectrogram_db.numpy(), cmap='viridis')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time Frame')
plt.title('Spectrogram')
# plt.ylim(0, 10)
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Parameters
fs = 30  # Sampling frequency in Hz
t = torch.linspace(0, 1, 3600)
x = torch.sin(2 * np.pi * 5 * t) * torch.sin(2 * np.pi * 2 * t)

# STFT parameters
n_fft = 256  # Number of FFT points
hop_length = fs  # Number of samples between successive frames
win_length = n_fft  # Window length
window = torch.hann_window(win_length)

# Compute the STFT
stft = torch.stft(x, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, return_complex=True)
print('stft.shape', stft.shape)

# Compute the magnitude spectrogram
spectrogram = torch.abs(stft)
print('spectrogram.shape', spectrogram.shape)

# Convert to dB scale (optional)
spectrogram_db = 10 * torch.log10(spectrogram + 1e-10)  # Add a small value to avoid log(0)

# Calculate frequency and time bins
frequencies = torch.fft.fftfreq(n_fft, 1 / fs)[:n_fft//2 + 1]  # Only keep non-negative frequencies
print('frequencies.shape', frequencies.shape)
times = torch.arange(stft.size(1)) * hop_length / fs
print('times.shape', times.shape)

# Plot the spectrogram
plt.figure(figsize=(10, 6))
plt.pcolormesh(times.numpy(), frequencies.numpy(), spectrogram_db.numpy(), cmap='viridis', shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram')
plt.show()

In [ ]:
import torch
import torchaudio
import torchaudio.transforms as T

In [ ]:
# Create the Spectrogram transform
spectrogram_transform = T.Spectrogram(
    n_fft=200,        # number of FFT bins
    win_length=None,  # window size, default to n_fft
    hop_length=30,   # hop length between frames
    power=2.0         # power to scale the magnitude
)

# Apply the transform to the waveform
spectrogram = spectrogram_transform(gt_signal_torch)
print(spectrogram.shape)